In [1]:
import numpy as np
from enum import StrEnum
from itertools import pairwise

In [2]:
# loading map data into numpy array
with open('map.txt') as f:
    map = f.readlines()

# map = """....#.....
# .........#
# ..........
# ..#.......
# .......#..
# ..........
# .#..^.....
# ........#.
# #.........
# ......#...""".split('\n')

map = [x.strip() for x in map]
map = [list(x) for x in map]
map = np.array(map)

In [3]:
# directions
class MapMarkers(StrEnum):
    UP = 'U'
    RIGHT = 'R'
    LEFT = 'L'
    DOWN = 'D'
    VISITED = 'X'
    START = '^'
    OBSTACLE = '#'

In [4]:
def get_next_coordinates(
    map: np.array, current_coordinates: tuple[int], direction: str
) -> tuple[tuple[int, int], str, bool]:
    direction_map = {
        MapMarkers.UP: (-1, 0),
        MapMarkers.DOWN: (1, 0),
        MapMarkers.LEFT: (0, -1),
        MapMarkers.RIGHT: (0, 1),
    }

    next_direction_map = {
        MapMarkers.UP: MapMarkers.RIGHT,
        MapMarkers.RIGHT: MapMarkers.DOWN,
        MapMarkers.DOWN: MapMarkers.LEFT,
        MapMarkers.LEFT: MapMarkers.UP,
    }

    while True:
        delta = direction_map[direction]
        next_coordinates = (
            current_coordinates[0] + delta[0],
            current_coordinates[1] + delta[1],
        )

        # check for out of bounds:
        if (
            next_coordinates[0] < 0
            or next_coordinates[1] < 0
            or next_coordinates[0] >= map.shape[0]
            or next_coordinates[1] >= map.shape[1]
        ):
            return next_coordinates, direction, True

        if map[next_coordinates] == MapMarkers.OBSTACLE:
            direction = next_direction_map[direction]
            continue
        else:
            return next_coordinates, direction, False

In [5]:
start = np.where(map == MapMarkers.START)
start = int(start[0][0]), int(start[1][0])

In [6]:
# find starting coordinates
current_coordindates = start
direction = MapMarkers.UP
steps = [(current_coordindates, direction)]

while True:
    next_coordinates, direction, oob = get_next_coordinates(
        map, current_coordindates, direction
    )
    steps.append((next_coordinates, direction))
    # checking if coordinates are outside of the map
    if oob:
        break
    else:
        current_coordindates = next_coordinates

In [7]:
len(steps)

6006

In [8]:
map_steps = map.copy()

for step, _ in steps[:-1]:
    map_steps[step] = MapMarkers.VISITED

for row in map_steps:
    print(''.join(row))

....#.................#......................#..........................#..................#....##..#....X......#.................
...................................#...............................#......#..#...........................X...#....................
..........................#................#......##.....#.....................................#.........X.....#..#...............
..........................XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#.X........................
...................#....#.X.......................#..............#.....#..............................XXXXXXXXXXXXXXXXXXXXXXXX#...
.....#.............XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#.............X....
...................X......X..............................#....................#..#............#.......XX.X....X#.............X....
..............#....X......X...#..........................XXXXXXXXXXXXXXXXXXXXXXXXXX

In [9]:
np.unique(map_steps, return_counts=True)

(array(['#', '.', 'X'], dtype='<U1'), array([  785, 10654,  5461]))

In [10]:
# subtracting 1 because we don't want to count the starting position
len({c for c, _ in steps}) - 1

5461

In [18]:
# Part 2
# need to check where in the steps I need to put an obstacle to create an infinite loop
# visiting the same coordinates with the same direction should imply an infinite loop

infinite_loops = []
path = steps[:-1]
path_visited = set()

for (coordinates, direction), (next_step, _) in pairwise(path):
    path_visited.add((coordinates, direction))
    current_path = set() | path_visited

    # temporarily add obstacle
    original_value = map[next_step]
    map[next_step] = MapMarkers.OBSTACLE

    while True:
        next_coordinates, next_direction, oob = get_next_coordinates(
            map, coordinates, direction
        )

        # if we have visited the next coordinates with the same direction, we have an infinite loop
        if (next_coordinates, next_direction) in current_path:
            infinite_loops.append(next_step)
            break

        # checking if coordinates are outside of the map
        if oob:
            break

        current_path.add((next_coordinates, next_direction))
        coordinates = next_coordinates
        direction = next_direction

    # remove obstacle
    map[next_step] = original_value

In [19]:
print(len(infinite_loops))
print(len(set(infinite_loops)))

2079
1972


In [14]:
# Part 2
# this time always starting from the beginning

infinite_loops = []
path = steps[:-1]
start_coordinates = steps[0][0]
start_direction = steps[0][1]

for _, (next_step, _) in pairwise(path):
    coordinates = start_coordinates
    direction = start_direction
    current_path = set((start_coordinates, start_direction))

    # temporarily add obstacle
    original_value = map[next_step]
    map[next_step] = MapMarkers.OBSTACLE

    while True:
        next_coordinates, next_direction, oob = get_next_coordinates(
            map, coordinates, direction
        )

        # if we have visited the next coordinates with the same direction, we have an infinite loop
        if (next_coordinates, next_direction) in current_path:
            infinite_loops.append(next_step)
            break

        # checking if coordinates are outside of the map
        if oob:
            break

        current_path.add((next_coordinates, next_direction))
        coordinates = next_coordinates
        direction = next_direction

    # remove obstacle
    map[next_step] = original_value

print(infinite_loops)

[(42, 102), (42, 103), (42, 109), (51, 114), (54, 114), (56, 114), (63, 114), (96, 114), (106, 97), (106, 85), (106, 73), (106, 71), (106, 45), (104, 43), (103, 43), (74, 43), (70, 49), (70, 78), (73, 80), (79, 80), (80, 78), (80, 76), (80, 73), (80, 70), (80, 67), (80, 58), (80, 42), (80, 33), (80, 28), (80, 20), (52, 49), (52, 70), (52, 76), (52, 78), (52, 81), (52, 105), (56, 106), (61, 106), (63, 106), (65, 106), (66, 106), (70, 106), (81, 106), (88, 99), (88, 96), (88, 90), (73, 88), (53, 88), (51, 88), (41, 88), (40, 104), (40, 107), (51, 112), (61, 112), (63, 112), (65, 112), (66, 112), (84, 112), (84, 107), (84, 102), (84, 99), (84, 96), (84, 90), (84, 89), (84, 87), (84, 78), (84, 76), (84, 73), (84, 70), (84, 55), (74, 47), (69, 47), (55, 49), (55, 53), (55, 76), (55, 78), (55, 81), (61, 93), (70, 93), (73, 93), (79, 93), (81, 93), (83, 93), (85, 93), (87, 93), (89, 93), (98, 93), (100, 93), (107, 93), (113, 90), (113, 85), (113, 73), (113, 71), (113, 55), (113, 42), (107, 40

In [15]:
len(infinite_loops)

2003

In [16]:
len(set(infinite_loops))

1836